In [1]:
import os

import sys
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN\\src")
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN")
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN\\venv")

path = os.getcwd()

curr_path = path.replace('\\', '/')
head_place = curr_path.find('GIEBN')
main_path = curr_path[:head_place]+'GIEBN'
main_path

'c:/Users/User/PycharmProjects/GIEBN'

In [2]:
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, GradientBoostingRegressor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import KBinsDiscretizer, FunctionTransformer, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline, make_union
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor


from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel, f_regression

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, RFE

from auxiliary.auxiliary import construct_by_kmeans

from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.models import BayesianNetwork
import yodo.yodo
import matplotlib.pyplot as plt
from pgmpy.estimators import PC, BicScore, K2Score
from pgmpy.inference import VariableElimination
from bamt.networks.discrete_bn import DiscreteBN
from bamt_inherited.networks.discrete import DiscreteBNGI
import shap
from sklearn import metrics as m
from tqdm import tqdm
from itertools import product
import optuna
from copy import copy

In [3]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [4]:
%run ../common_code.ipynb

In [9]:
#https://www.kaggle.com/datasets/camnugent/california-housing-prices
data = pd.read_csv("../../datasets/Bike_Sharing_Demand.csv")

n = data.shape[0]
target = "count"
features = ['month', 'hour', 'temp', 'feel_temp', 'humidity', 'windspeed']
data = data[features+[target]]
n_features = len(features)
data.head(10)

,month,hour,temp,feel_temp,humidity,windspeed,count
0,1,0,9.84,14.395,0.81,0.0000,16.0
1,1,1,9.02,13.635,0.80,0.0000,40.0
2,1,2,9.02,13.635,0.80,0.0000,32.0
3,1,3,9.84,14.395,0.75,0.0000,13.0
4,1,4,9.84,14.395,0.75,0.0000,1.0
5,1,5,9.84,12.880,0.75,6.0032,1.0
6,1,6,9.02,13.635,0.80,0.0000,2.0
7,1,7,8.20,12.880,0.86,0.0000,3.0
8,1,8,9.84,14.395,0.75,0.0000,8.0
9,1,9,13.12,17.425,0.76,0.0000,14.0


In [10]:
data.describe()

,month,hour,temp,feel_temp,humidity,windspeed,count
count,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
mean,6.537775,11.546752,20.376474,23.788755,0.627229,12.736540,189.463088
std,3.438776,6.914405,7.894801,8.592511,0.192930,8.196795,181.387599
min,1.000000,0.000000,0.820000,0.000000,0.000000,0.000000,1.000000
25%,4.000000,6.000000,13.940000,16.665000,0.480000,7.001500,40.000000
50%,7.000000,12.000000,20.500000,24.240000,0.630000,12.998000,142.000000
75%,10.000000,18.000000,27.060000,31.060000,0.780000,16.997900,281.000000
max,12.000000,23.000000,41.000000,50.000000,1.000000,56.996900,977.000000


In [168]:
data_corr = add_extra_features(data, features, extra_size=0.75, mode='prod')

In [169]:
data_dict_corr = {}

data_tv, data_dict_corr['test'] = train_test_split(data_corr, test_size=0.15, random_state=42)

data_dict_corr['train'], data_dict_corr['val'] = train_test_split(data_tv, test_size=0.15, random_state=42)

features_corr = list(data_corr.columns.difference([target]))

In [170]:
features_corr

['extra_1',
 'extra_10',
 'extra_11',
 'extra_12',
 'extra_13',
 'extra_14',
 'extra_15',
 'extra_16',
 'extra_17',
 'extra_18',
 'extra_2',
 'extra_3',
 'extra_4',
 'extra_5',
 'extra_6',
 'extra_7',
 'extra_8',
 'extra_9',
 'feel_temp',
 'hour',
 'humidity',
 'month',
 'temp',
 'windspeed']

In [171]:
def objective(trial, features, target, data_dict):
    n_estimators= trial.suggest_int("n_estimators", low=params_net['n_estimators'][0], high=params_net['n_estimators'][1], step=1)
    max_depth= trial.suggest_int("max_depth", low=params_net['max_depth'][0], high=params_net['max_depth'][1], step=1)
    learning_rate=trial.suggest_float("learning_rate", low=params_net['learning_rate'][0], high=params_net['learning_rate'][1],log=True)
    
    regr = GradientBoostingRegressor(random_state=42, n_estimators=n_estimators,
                                     max_depth=max_depth, learning_rate=learning_rate)
    regr.fit(data_dict['train'][features], data_dict['train'][target])
    return score_regr(regr, data_dict['val'][features], data_dict['val'][target])

params_net = {
    'n_estimators': [10, 100],
    'max_depth': [3, 10],
    'learning_rate':[1e-5, 1]
}


# Initial model for error calculation

In [172]:
best_base_params = optunize_model(objective, features_corr, target, data_dict_corr)

  0%|          | 0/30 [00:00<?, ?it/s]

In [173]:
best_base_params

{'n_estimators': 100, 'max_depth': 7, 'learning_rate': 0.13399296762318771}

In [174]:
model_corr = GradientBoostingRegressor(random_state=42, **best_base_params)
#model = LogisticRegression()
model_corr.fit(data_dict_corr['train'][features_corr], data_dict_corr['train'][target])

data_errors_corr = measure_error(data_dict_corr['val'], model_corr, features_corr, target, error_col_name='ape_error', task='regr')

In [175]:
n_features_corr = len(features_corr)
disc_data_corr, pipeline_data_corr = discretize(data_errors_corr, 
                                      features_corr+["ape_error"], icat=None, icont=list(range(n_features_corr+1)))

categories_corr = extract_categories(disc_data_corr)
encoder = OneHotEncoder(sparse=False)
disc_data_encoded_corr = pd.DataFrame(encoder.fit_transform(X=disc_data_corr), columns=categories_corr, dtype='uint8')
disc_data_encoded_corr.head(1)

,month_0,month_1,month_2,hour_0,hour_1,hour_2,temp_0,temp_1,temp_2,feel_temp_0,...,extra_16_2,extra_17_0,extra_17_1,extra_17_2,extra_18_0,extra_18_1,extra_18_2,ape_error_0,ape_error_1,ape_error_2
0,0,0,1,1,0,0,0,1,0,0,...,0,1,0,0,1,0,0,0,0,1


In [176]:
disc_data_corr['ape_error'].value_counts()

ape_error
1    886
2    665
0    665
Name: count, dtype: int64

In [179]:
feat_to_grad_corr = {}
for feat in features_corr+['ape_error']:
    feat_to_grad_corr[feat]=[grad for grad in disc_data_encoded_corr.columns if feat in grad]


metric_dict = {
    'mape': lambda x, y: m.mean_absolute_percentage_error(x,y)*100,
    'r2_score': lambda x, y: m.r2_score(x, y)*100
}

gbn_fs = DiscreteBNGI('')

gbn_training_pipeline(gbn_fs, disc_data_encoded_corr, 'ape_error', categories_corr, score=BicScore)

2025-05-04 18:14:01,824 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-04 18:14:01,824 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-04 18:14:01,825 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-04 18:14:01,826 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-04 18:14:01,827 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-04 18:14:01,828 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-04 18:14:01,828 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-04 18:14:01,830 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-04 18:14:01,831 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-04 18:14:01,832 | ERROR    | GraphUtil

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [180]:
gbn_fs.get_info()

,name,node_type,data_type,parents,parents_types
0,extra_12_1,Discrete,disc,[],[]
1,extra_9_1,Discrete,disc,[extra_12_1],[disc]
2,extra_12_2,Discrete,disc,[extra_9_1],[disc]
3,extra_9_2,Discrete,disc,[extra_12_2],[disc]
4,extra_16_2,Discrete,disc,[extra_12_2],[disc]
...,...,...,...,...,...
70,extra_4_1,Discrete,disc,[extra_10_1],[disc]
71,extra_6_2,Discrete,disc,"[hour_2, humidity_1, extra_14_2]","[disc, disc, disc]"
72,extra_11_0,Discrete,disc,"[month_0, extra_9_0, extra_12_0]","[disc, disc, disc]"
73,extra_11_2,Discrete,disc,"[month_0, month_1, temp_2, feel_temp_0]","[disc, disc, disc, disc]"


In [181]:
_df=gbn_fs.get_info()
_df[_df['name'].apply(lambda x: 'ape_error' in x.name)]

,name,node_type,data_type,parents,parents_types
39,ape_error_0,Discrete,disc,"[hour_0, extra_15_2]","[disc, disc]"
40,ape_error_1,Discrete,disc,[hour_0],[disc]
58,ape_error_2,Discrete,disc,"[hour_0, humidity_2]","[disc, disc]"


# Univariate Feature selector

In [182]:
univ_sel = SelectKBest(f_regression, k=len(features_corr))
univ_sel.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
df_val_corr_univ = pd.DataFrame({'Feature': univ_sel.feature_names_in_, 'Metric value': univ_sel.pvalues_})

# RFE on decision tree

In [183]:
rfe_sel = RFE(estimator=DecisionTreeRegressor(random_state=42, max_depth=10,), n_features_to_select=1)
#rfe_sel = RFE(estimator=RandomForestClassifier(random_state=42, max_depth=10, n_estimators=50), n_features_to_select=1, )
rfe_sel.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
df_val_corr_rfe = pd.DataFrame({'Feature': rfe_sel.feature_names_in_, 'Metric value': rfe_sel.ranking_})

# Lasso from Logistic Regression with L1 penalty

In [184]:
lsvc = Lasso(alpha=1e1, random_state=42)
lsvc.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
lasso_selector = SelectFromModel(lsvc, prefit=True)

In [185]:
lasso_support = [x for i, x in enumerate(features_corr) if lasso_selector.get_support()[i]]
lasso_support

['extra_1',
 'extra_10',
 'extra_11',
 'extra_12',
 'extra_13',
 'extra_14',
 'extra_16',
 'extra_17',
 'extra_2',
 'extra_4',
 'extra_6',
 'extra_7',
 'extra_9',
 'feel_temp',
 'hour',
 'temp']

## GBN selector

In [186]:
df_val_corr_bn=error_based_importance_features_bn(gbn_fs, 'ape_error', features_corr, feat_to_grad_corr).fillna(0)

100%|██████████| 24/24 [00:06<00:00,  3.82it/s]


## GBN selector run

In [187]:
gbn_res = elimination_experiment(model_corr, data_dict_corr, metric_dict, df_val_corr_bn, direction='asc', task='regr')

100%|██████████| 24/24 [03:21<00:00,  8.40s/it]


In [188]:
gbn_res.head()

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,24,-,101.4,68.005,0,0
1,23,extra_18,103.005,67.905,1.583,-0.148
2,22,extra_12,103.23,67.768,1.804,-0.349
3,21,windspeed,102.131,68.048,0.721,0.064
4,20,extra_3,102.037,67.939,0.628,-0.097


In [189]:
get_best_record(gbn_res, 'mape', mode='min')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
5,19,extra_17,96.446,69.081,-4.886,1.582


In [190]:
get_best_record(gbn_res, 'r2_score')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
7,17,extra_5,97.016,69.192,-4.324,1.745


In [191]:
df_corr_excluded_asc_univ=elimination_experiment(model_corr, data_dict_corr, metric_dict,
                       df_val_corr_univ, 'asc', task='regr')
df_corr_excluded_asc_univ.head()

100%|██████████| 24/24 [04:32<00:00, 11.36s/it]


,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,24,-,101.884,67.969,0,0
1,23,extra_1,102.344,67.963,0.451,-0.009
2,22,extra_15,105.49,68.421,3.539,0.664
3,21,temp,104.794,68.543,2.857,0.845
4,20,extra_9,103.984,68.291,2.061,0.474


In [192]:
get_best_record(df_corr_excluded_asc_univ, 'mape', mode='min')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
8,16,hour,101.418,68.109,-0.457,0.206


In [193]:
get_best_record(df_corr_excluded_asc_univ, 'r2_score')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
3,21,temp,104.794,68.543,2.857,0.845


In [194]:
df_corr_excluded_rfe=elimination_experiment(model_corr, data_dict_corr, metric_dict,
                       df_val_corr_rfe, 'desc', task='regr')
df_corr_excluded_rfe.head()

100%|██████████| 24/24 [03:40<00:00,  9.17s/it]


,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,24,-,102.905,67.891,0,0
1,23,feel_temp,102.323,67.864,-0.566,-0.04
2,22,windspeed,102.264,68.009,-0.623,0.174
3,21,extra_16,101.085,68.22,-1.768,0.484
4,20,temp,100.616,68.19,-2.224,0.44


In [195]:
get_best_record(df_corr_excluded_rfe, 'mape', mode='min')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
9,15,extra_3,99.769,68.327,-3.048,0.642


In [196]:
get_best_record(df_corr_excluded_rfe, 'r2_score')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
10,14,month,100.718,68.536,-2.126,0.95


In [197]:
lasso_support

['extra_1',
 'extra_10',
 'extra_11',
 'extra_12',
 'extra_13',
 'extra_14',
 'extra_16',
 'extra_17',
 'extra_2',
 'extra_4',
 'extra_6',
 'extra_7',
 'extra_9',
 'feel_temp',
 'hour',
 'temp']

In [198]:
single_run(model_corr, data_dict_corr, lasso_support, target, metric_dict, task='regr')

[101.332, 68.662]

In [ ]:
#logging mape (probably woth to be negative mape)
0.5 random, base 18.467, gbn 17.877, univ 18.467, rfe 18.103 , lasso: 17.89
0.75 random, base 19.137, gbn 18.346, univ 19.137, rfe 18.239 , lasso: 18.338
0.5 noisy, base 18.337, gbn 17.927, univ 18.337, rfe  18.109, lasso: 18.221
0.75 noisy, base 18.442, gbn 17.960, univ 18.442, rfe  18.185, lasso: 18.586
0.5 prod, base 17.865, gbn 17.865, univ 17.865, rfe 17.802, lasso: 17.762
0.75 prod, base 18.442, gbn 18.417, univ 18.382, rfe 18.029, lasso: 18.585


In [ ]:
# logging r2_score
0.5 random, base 81.675, gbn 82.667, univ 81.675, rfe  82.27, lasso: 82.616 
0.75 random, base 80.482, gbn 81.832, univ 80.619, rfe  82.188, lasso: 81.806
0.5 noisy, base 81.813, gbn 82.479, univ 81.863, rfe  82.047, lasso: 81.88
0.75 noisy, base 81.637, gbn 82.039, univ 81.596, rfe  81.97, lasso: 81.392
0.5 prod, base 81.97, gbn 81.97, univ 82.176, rfe 82.438, lasso: 82.363
0.75 prod, base 81.637, gbn 81.637, univ 81.704, rfe 82.364, lasso: 80.941


In [ ]:
# #logging mape
# 0.5 random, base 17.877, gbn 80.075, univ 79.953, rfe 79.953, lasso: 79.944
# 0.75 random, base 80.119, gbn 80.55	, univ 80.119, rfe 80.119, lasso: 80.242

# 0.5 noisy base 80.42, gbn 80.58	, univ 80.42, rfe 80.578, lasso: 80.515
# 0.75 noisy base 80.264, gbn 80.311, univ 80.264, rfe 80.264, lasso: 80.633

# 0.5 prod base 80.007, gbn 80.203	, univ 80.022, rfe 80.448, lasso: 80.422
# 0.75 prod base 80.36, gbn 80.422	, univ 80.549, rfe 80.542, lasso: 80.316
